Below is TOPSIS method


In [25]:
import numpy as np
import pandas as pd

In [26]:
a = np.array([[1,2],[3,4]])
a = a.T 


In [27]:
def TOPSIS(matric,weights, impacts):
    """
    Perform TOPSIS ranking.
    
    Parameters
    ----------
    matrix : 2D array-like
        Decision matrix with shape (m, n) — m alternatives, n criteria.
    weights : 1D array-like
        Importance weights for each criterion (should sum to 1).
    criteria_types : list of str
        Each element is 'benefit' or 'cost' indicating the type of criterion.
    
    Returns
    ----------
    scores : ndarray
        Closeness coefficient for each alternative (higher = better).
    rankings : ndarray
        Indices of alternatives ranked from best to worst.
    """
    # Avoid wrong imports
    
    if sum(weights) != 1:
        raise ValueError("Weights must sum to 1.")
    if len(weights) != matric.shape[1]:
        raise ValueError("Number of weights must match number of criteria.")
    
    # Step 1: Normalize the decision matrix
    norm_matric = matric / np.sqrt((matric**2).sum())
    
    # Step 2: Multiply by weights
    weighted_matric = norm_matric * weights

    # Step 3: Determine ideal and negative-ideal solutions
    ideal_solution = []
    negative_ideal_solution = []
    for i in range(weighted_matric.shape[1]):
        if impacts[i] == '+':
            ideal_solution.append(weighted_matric[:, i].max())
            negative_ideal_solution.append(weighted_matric[:, i].min())
        else:
            ideal_solution.append(weighted_matric[:, i].min())
            negative_ideal_solution.append(weighted_matric[:, i].max())
    
    ideal_solution = np.array(ideal_solution)
    negative_ideal_solution = np.array(negative_ideal_solution)

    # Step 4: Calculate distances to ideal and negative-ideal solutions
    distance_to_ideal = np.sqrt(((weighted_matric - ideal_solution) ** 2).sum(axis=1))
    distance_to_negative_ideal = np.sqrt(((weighted_matric - negative_ideal_solution) ** 2).sum(axis=1))

    # Step 5: Calculate performance score
    performance_score = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)

    return performance_score

In [43]:
from asyncio import log


def entropy(infor):
    """

    Args:
        infor (np.array): input array

    Returns:
        float: entropy value
    """
    return np.sum(-infor * np.log(infor + np.finfo(float).eps)) / np.log(len(infor))
def cost_indicator(matric):
    """
    Args:
        matric (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: cost indicator matrix
    """
    ind = np.argmax(matric,axis = 1)
    maxi = matric[ind,:]
    print(maxi)
    print(np.sum(maxi-matric,axis = 0))
    matric = (maxi-matric)/np.sum(maxi-matric,axis = 0)
    return matric
def benefit_indicator(matric):
    """
    Args:
        matric (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: benefit indicator matrix
    """
    
    return matric/np.sum(matric,axis = 0)
def entropy_weights(matric,indicator):
    """
    Args:
        matric (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: entropy weights (1,n)
    """
    # Normalization
    #matric = matric / matric.sum(axis=0)
    matric = indicator(matric)
    print(matric)
    matric = np.where(matric == 0, np.finfo(float).eps, matric)
    # Entropy calculation
    n_criteria = matric.shape[1]
    
    entropy_values = np.array(list(map(entropy, matric.T)))
   
    print(entropy_values)
    # Degree of diversification
    diver = 1- entropy_values
    weights = diver / np.sum(diver)
    return weights
    

entropy_weights(np.array([[10,200,0.5],
                          [20,300,0.7],
                          [30,250,0.9]]),benefit_indicator)

    
    
    

[[0.16666667 0.26666667 0.23809524]
 [0.33333333 0.4        0.33333333]
 [0.5        0.33333333 0.42857143]]
[0.92061984 0.98778124 0.97488309]


array([0.6801148 , 0.10468808, 0.21519712])

In [29]:
def AHP(weights_matrix):
    """
    Perform AHP to derive weights from pairwise comparison matrix.
    
    Parameters
    ----------
    weights_matrix : 2D array-like
        Pairwise comparison matrix (n x n).
    
    Returns
    ----------
    weights : ndarray
        Derived weights for each criterion.
    """
    eigvals, eigvecs = np.linalg.eig(weights_matrix)
    max_index = np.argmax(eigvals)
    weights = np.real(eigvecs[:, max_index])
    weights = weights / weights.sum()
    return weights

In [30]:
AHP(np.array([[1, 3, 5],
             [1/3, 1, 2],
             [1/5, 1/2, 1]]))

array([0.64832901, 0.22965079, 0.12202019])

In [ ]:
# indicator = [Energy consumption, emmision,waste management,water consumption, temperture]




NameError: name 'de' is not defined

**问题1：理解问题 - 环境影响指标体系**
**思路**
**建立三级指标体系：**


- 一级指标：能源、交通、废弃物、水资源、气候适应性


- 二级指标：细化的可测量指标 


- 三级指标：具体数据采集点 


**关键指标** 


1. **energy consumption**


    - 可再生能源比例（%）  


    - 场馆能源效率（kWh/座位）   


    - 电网清洁度（碳强度 gCO₂/kWh）   


    - 当地太阳能/风能潜力   


2. **carbon emission by transportation**
    - 平均观众出行距离（km）   


    - 公共交通覆盖率（%）   


    - 机场碳排放效率   


    - 电动车充电基础设施密度   


3. **waste management**
    - 回收率（%）   


    - 人均废弃物产生量（kg/人）   


    - 堆肥设施可用性   


    - 一次性塑料使用政策   


4. **water usage**
    - 水资源压力指数   


    - 雨水回收系统   


    - 节水技术应用率   


5. **climate**


    - 热岛效应强度   